In [2]:
import tensorflow as tf
import numpy as np
import time

if __name__ == "__main__":

    size_array = [8,16,32,64,128,256,512,1024,2048,4096,8192]
    loop_time = 10000
    for size in size_array:
        with tf.device("/cpu:0"):
            c = tf.Variable(tf.convert_to_tensor(np.eye(size),dtype=tf.float64),name='c-cpu')
            a = tf.constant(np.random.normal(loc=0.0,scale=0.01,size=(size,size)),name='a-cpu')
              

            i = tf.constant(1,name='i-cpu')
            n = tf.constant(loop_time,name='n-cpu')

            def cond(i,n,c,a):
                return i<n

            def loop(i,n,c,a):
                i=i+1
                c=c*a
                return i,n,c,a

            cpu_master = tf.while_loop(cond,loop,[i,n,c,a],name='loop-cpu')

        with tf.device("/gpu:0"):
            C = tf.Variable(tf.convert_to_tensor(np.eye(size),dtype=tf.float64),name='C-GPU')
            A = tf.constant(np.random.normal(loc=0.0,scale=0.01,size=(size,size)),name='A-GPU')


            I = tf.constant(1,name='I-GPU')
            N = tf.constant(loop_time,name='N-GPU')

            def COND(I,N,C,A):
                return I<N

            def LOOP(I,N,C,A):
                I=I+1
                C=C*A
                return I,N,C,A

            GPU_MASTER = tf.while_loop(COND,LOOP,[I,N,C,A],name='LOOP-GPU')

        init = tf.global_variables_initializer()

        with tf.Session() as sess:
            sess.run(init)

            t1=time.time()
            sess.run(cpu_master)


            t2=time.time()
            sess.run(GPU_MASTER)

            t3=time.time()
            print ("matrix size: %f" %size)
            print ("  CPU : %f s" %(t2-t1))
            print ("  GPU : %f s" %(t3-t2))
            print ("CPU : GPU = %f" %( (t2-t1)/(t3-t2) ) )
            print ("---------------")

matrix size: 8.000000
  CPU : 0.135325 s
  GPU : 0.131681 s
CPU : GPU = 1.027673
---------------
matrix size: 16.000000
  CPU : 0.136366 s
  GPU : 0.123661 s
CPU : GPU = 1.102741
---------------
matrix size: 32.000000
  CPU : 0.132178 s
  GPU : 0.133530 s
CPU : GPU = 0.989880
---------------
matrix size: 64.000000
  CPU : 0.130405 s
  GPU : 0.133845 s
CPU : GPU = 0.974303
---------------
matrix size: 128.000000
  CPU : 0.177283 s
  GPU : 0.132360 s
CPU : GPU = 1.339395
---------------
matrix size: 256.000000
  CPU : 0.302699 s
  GPU : 0.121103 s
CPU : GPU = 2.499524
---------------
matrix size: 512.000000
  CPU : 0.464176 s
  GPU : 0.179137 s
CPU : GPU = 2.591181
---------------
matrix size: 1024.000000
  CPU : 4.152907 s
  GPU : 0.788786 s
CPU : GPU = 5.264934
---------------
matrix size: 2048.000000
  CPU : 32.472494 s
  GPU : 3.101184 s
CPU : GPU = 10.470998
---------------
matrix size: 4096.000000
  CPU : 138.045926 s
  GPU : 11.323462 s
CPU : GPU = 12.191141
---------------
matrix